# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [6]:
# Your solution goes here

import requests
from bs4 import BeautifulSoup
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,"

headers = { "User-Agent": "Chrome/91.0.4472.114" }
response = requests.get(url, headers=headers)

response #successfully load the webpage

<Response [200]>

In [100]:
movies = soup.find_all("div",attrs={"class":"ipc-metadata-list-summary-item__c"})
len(movies)

25

In [92]:
from bs4 import BeautifulSoup
soup =BeautifulSoup(response.content)
movies[0].find_all("h3",attrs={"ipc-title__text"})[0].get_text()

#extracting the movie title and position in the page as well

'1. O Silêncio dos Inocentes'

In [93]:
movies[0].find_all("span",attrs={"sc-b189961a-8 kLaxqf dli-title-metadata-item"})[0].get_text()
#extracting the release year

'1991'

In [95]:
movies[0].find_all("span",attrs={"ipc-rating-star--rating"})[0].get_text()
#extracting the movie rating

'8.6'

In [96]:
movies[0].find_all("span",attrs={"sc-b189961a-8 kLaxqf dli-title-metadata-item"})[1].get_text()
#extracting the movie runtime

'1h 58m'

In [97]:
movies[0].find_all("div",attrs={"ipc-html-content-inner-div"})[0].get_text()
#extracting the description

'A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.'

In [98]:
movies[0].find_all("span",attrs={"ipc-rating-star--voteCount"})[0].get_text().strip('\xa0()')
#extracting the votes

'1.6M'

In [102]:
#get all item data
movies_dict = {}
index = 0

for movie in movies:
    title = movie.find_all("h3",attrs={"ipc-title__text"})[0].get_text()
    year = movie.find_all("span",attrs={"sc-b189961a-8 kLaxqf dli-title-metadata-item"})[0].get_text()
    rating = movie.find_all("span",attrs={"ipc-rating-star--rating"})[0].get_text()
    runtime = movie.find_all("span",attrs={"sc-b189961a-8 kLaxqf dli-title-metadata-item"})[1].get_text()
    description = movie.find_all("div",attrs={"ipc-html-content-inner-div"})[0].get_text()
    votes = movie.find_all("span",attrs={"ipc-rating-star--voteCount"})[0].get_text().strip('\xa0()')

    movies_dict[index] = {"title":title,
                          "year":year,
                          "rating":rating,
                          "runtime":runtime,
                          "description":description,
                          "votes":votes}

    index+=1

movies_dict

{0: {'title': '1. O Silêncio dos Inocentes',
  'year': '1991',
  'rating': '8.6',
  'runtime': '1h 58m',
  'description': 'A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.',
  'votes': '1.6M'},
 1: {'title': '2. Tudo Bons Rapazes',
  'year': '1990',
  'rating': '8.7',
  'runtime': '2h 25m',
  'description': 'The story of Henry Hill and his life in the mafia, covering his relationship with his wife Karen and his mob partners Jimmy Conway and Tommy DeVito.',
  'votes': '1.3M'},
 2: {'title': '3. O Meu Primo Vinny',
  'year': '1992',
  'rating': '7.6',
  'runtime': '2h',
  'description': 'Two New Yorkers accused of murder in rural Alabama while on their way back to college call in the help of one of their cousins, a loudmouth lawyer with no trial experience.',
  'votes': '143K'},
 3: {'title': '4. Exterminador Implacável 2 - O Dia do Julgamento',
  'year': '1991',
  'rating':

In [103]:
#creating a data set with the data from imdb:

import pandas as pd
movie_df = pd.DataFrame.from_dict(movies_dict,orient="index")
movie_df

,title,year,rating,runtime,description,votes
0,1. O Silêncio dos Inocentes,1991,8.6,1h 58m,A young F.B.I. cadet must receive the help of ...,1.6M
1,2. Tudo Bons Rapazes,1990,8.7,2h 25m,The story of Henry Hill and his life in the ma...,1.3M
2,3. O Meu Primo Vinny,1992,7.6,2h,Two New Yorkers accused of murder in rural Ala...,143K
3,4. Exterminador Implacável 2 - O Dia do Julgam...,1991,8.6,2h 17m,"A cyborg, identical to the one who failed to k...",1.2M
4,5. Mulheres do Sul,1991,7.7,2h 10m,"Evelyn, an ordinary housewife, visits a nursin...",85K
5,6. Imperdoável,1992,8.2,2h 10m,Retired Old West gunslinger William Munny relu...,439K
6,7. Cães Danados,1992,8.3,1h 39m,When a simple jewelry heist goes horribly wron...,1.1M
7,8. Danças com Lobos,1990,8.0,3h 1m,"Lieutenant John Dunbar, assigned to a remote w...",294K
8,9. Sozinho em Casa,1990,7.7,1h 43m,"An eight-year-old troublemaker, mistakenly lef...",658K
9,10. JFK,1991,8.0,3h 9m,New Orleans District Attorney Jim Garrison dis...,171K


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here